In [139]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [140]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
df[393:396]

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
393,24/11/2020 19:15:02,30.0,Male,Moscow,Software Engineer,5,0,Middle,C,C/C++,14712.0,0,30,Full-time employee,Unlimited contract,Russian,101-1000,Product
394,24/11/2020 19:15:49,33.0,Male,Berlin,Product Manager,5,5,Senior,NaN,Python,70000.0,800,30,Full-time employee,NaN,German,101-1000,Product
395,24/11/2020 19:19:30,35.0,Male,Berlin,QA Engineer,11,10,Senior,Java,"Python, Javascript / Typescript, .NET, Java / ...",74400.0,NaN,30,Full-time employee,Unlimited contract,English,101-1000,Product


In [142]:
#df.info()
df["Other technologies/programming languages you use often"].value_counts()

Javascript / Typescript                                                                   44
Python                                                                                    37
SQL                                                                                       31
AWS, Docker                                                                               16
Kotlin                                                                                    15
                                                                                          ..
Python, Ruby, Java / Scala, Go, Rust, AWS, Kubernetes, Docker, Terraform, helm, packer     1
Kotlin, SQL, AWS, Kubernetes, Docker                                                       1
Python, R, Go                                                                              1
C/C++, PHP, SQL                                                                            1
Python, AWS, Kubernetes, Docker, terraform, ansible                   

In [143]:
df.sample(20)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
1225,06/01/2021 21:20:44,24.0,Male,Berlin,Frontend Developer,5,2,Senior,TypeScript,Javascript / Typescript,68250.0,0,27,Full-time employee,Unlimited contract,English,101-1000,Product
987,30/11/2020 19:28:15,34.0,Male,Berlin,ML Engineer,3,3,Senior,Python,AWS,70000.0,NaN,30,Full-time employee,Temporary contract,English,101-1000,Product
992,30/11/2020 21:19:16,35.0,Female,Munich,Data Scientist,5,2,Senior,"SAS,SQL,Python","Python, SQL, SAS",75000.0,10000,30,Full-time employee,Unlimited contract,English,1000+,Bank
590,25/11/2020 13:55:16,29.0,Male,Stuttgart,Software Engineer,7,5,Middle,C++,NaN,70000.0,0,30,Full-time employee,Unlimited contract,English,51-100,Product
459,24/11/2020 22:52:26,38.0,Male,Berlin,Software Engineer,16,2,Senior,ABAP,"PHP, SQL, SAP / ABAP, Delphi",83000.0,11000,27,Full-time employee,Unlimited contract,English,1000+,Startup
913,29/11/2020 19:42:21,24.0,Male,Berlin,ML Engineer,3,NaN,NaN,NaN,NaN,90000.0,NaN,NaN,NaN,NaN,NaN,NaN,Startup
522,25/11/2020 10:44:20,28.0,Male,Berlin,Data Scientist,4,4,Middle,Python,"Java / Scala, R, SQL, Docker, Julia",70000.0,5500,28,Full-time employee,Unlimited contract,English,101-1000,Product
1003,30/11/2020 23:38:05,30.0,Male,Berlin,Software Engineer,9,1,Middle,"TypeScript, Kotlin","Kotlin, Javascript / Typescript, Rust",77000.0,0,26,Full-time employee,Unlimited contract,English,1000+,Product
163,24/11/2020 12:18:41,32.0,Male,Frankfurt,Network Engineer,10,2,Senior,Python,"Python, Docker",68500.0,6000,25,Full-time employee,Unlimited contract,English,101-1000,Product
910,29/11/2020 17:48:04,25.0,Male,Frankfurt,DevOps,2,2,Junior,Python,"Python, AWS, Kubernetes, Docker",65000.0,NaN,28,Full-time employee,Unlimited contract,English,1000+,Consulting / Agency


# Answers and Explainations
(Expand/modify as needed)

### Results

### Feature Selection Activities

### Hyperparameter Changes

In [144]:
# Drop rows with NaN values in the columns used for calculation
df = df.dropna(subset=['Yearly brutto salary (without bonus and stocks) in EUR', 'Yearly bonus + stocks in EUR'])

# Convert columns to numeric
df['Yearly brutto salary (without bonus and stocks) in EUR'] = pd.to_numeric(df['Yearly brutto salary (without bonus and stocks) in EUR'], errors='coerce')
df['Yearly bonus + stocks in EUR'] = pd.to_numeric(df['Yearly bonus + stocks in EUR'], errors='coerce')

# Recalculate 'TotalCompensation'
df['TotalCompensation'] = df['Yearly brutto salary (without bonus and stocks) in EUR'] + df['Yearly bonus + stocks in EUR']

# Find out the only person in City X
city = df['City'].value_counts()
city_once = city[city == 1].index
city_once = pd.DataFrame(city_once, columns=['City'])

# Remove the rows with only one person in City X
df = df[df['City'].isin(city_once['City']) == False]

# Imputation
df.dropna(inplace=True)

# Column Transformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# Define numeric features
numeric_features = ["Age", "Yearly brutto salary (without bonus and stocks) in EUR", "Yearly bonus + stocks in EUR"]

# Define transformers
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

# Define categorical features
categorical_features = [col for col in df.columns if df[col].dtype == 'object' and col != 'Timestamp']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')  # Handle unknown categories

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

# Split data into features and target variable
y = df['TotalCompensation']
X = df.drop(columns=["TotalCompensation"])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Build pipeline
pipeline_steps = [('pre', preprocessor), ('DT', DecisionTreeRegressor())]
pipe = Pipeline(pipeline_steps)

pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

y_pred = pipe.predict(X_test)

# R2 Score
r2 = r2_score(y_test, y_pred)
print(f"R2 Score: {r2}")

# RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

R2 Score: 0.9699068309374522
RMSE: 9117.952372665624


Accuracy of my model:
<ul>
    <li> R2 Score: 0.9699068309374522
    <li> RMSE: 9117.952372665624
    <li> I did create a Tree model, and the result was that features removed, such as only 1 person in a city, rows with null value

In [145]:
# Hyperparameters: Pipeline with GridSearchCV

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Create lists of potential values.  
non_numeric_features = ["Timestamp"]
tree_para = {'DT__min_samples_split': [2, 3, 4, 5, 6, 7], 'DT__max_depth': [3, 4, 5, 6, 7, 8]}

# Use the pipeline in GridSearchCV
clf = GridSearchCV(pipe, param_grid=tree_para, cv=5, n_jobs=-1) 
clf.fit(X_train, y_train.ravel())
best = clf.best_estimator_
print(best.score(X_test, y_test))
print(best)

0.95832220660143
Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age',
                                                   'Yearly brutto salary '
                                                   '(without bonus and stocks) '
                                                   'in EUR',
                                                   'Yearly bonus + stocks in '
                                                   'EUR']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
             

<li> I tried Pipeline with GridSearchCV, it allows to find the optimal combination of hyperparameters for the pipeline

In [146]:
# Use ExtraTreesRegressor model


from sklearn.ensemble import ExtraTreesRegressor
from sklearn.compose import TransformedTargetRegressor

# Drop rows with NaN values in the columns used for calculation
df = df.dropna(subset=['Yearly brutto salary (without bonus and stocks) in EUR', 'Yearly bonus + stocks in EUR'])

# Recalculate 'TotalCompensation'
df['TotalCompensation'] = df['Yearly brutto salary (without bonus and stocks) in EUR'] + df['Yearly bonus + stocks in EUR']

# Find out the only person in City X
city = df['City'].value_counts()
city_once = city[city == 1].index
city_once = pd.DataFrame(city_once, columns=['City'])

# Remove the rows with only one person in City X
df = df[df['City'].isin(city_once['City']) == False]

# Imputation
df.dropna(inplace=True)

# Define numeric features
numeric_features = ["Age", "Yearly brutto salary (without bonus and stocks) in EUR", "Yearly bonus + stocks in EUR"]

# Define transformers
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_features = [col for col in df.columns if df[col].dtype == 'object' and col != 'Timestamp']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')  # Handle unknown categories

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

# Split data into features and target variable
y = df['TotalCompensation']
X = df.drop(columns=["TotalCompensation"])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Build pipeline
model = ExtraTreesRegressor()
transformer = TransformedTargetRegressor(regressor=model, func=np.log1p, inverse_func=np.expm1)

pipeline_steps = [('pre', preprocessor), ('ETR', transformer)]
pipe = Pipeline(pipeline_steps)

# Fit the model
pipe.fit(X_train, y_train)


y_train_pred = pipe.predict(X_train)

# Predictions on the test set
y_test_pred = pipe.predict(X_test)

# R2 Score
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"Training R2 Score: {r2_train}")
print(f"Testing R2 Score: {r2_test}")

# RMSE (Root Mean Squared Error)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Training RMSE: {rmse_train}")
print(f"Testing RMSE: {rmse_test}")

Training R2 Score: 1.0
Testing R2 Score: 0.975001888651733
Training RMSE: 1.8036781405237188e-09
Testing RMSE: 6004.551773948243


Accuracy of my model (ExtraTreesRegressor and TransformedTargetRegressor):
<ul>
    <li> Training R2 Score: 1.0
    <li> Testing R2 Score: 0.975001888651733
    <li> Training RMSE: 1.8036781405237188e-09
    <li> Testing RMSE: 6004.551773948243
    <li> ExtraTreesRegressor helps to build multiple decision trees to improve overall performance
    <li> TransformedTargetRegressor: Using a logarithmic transformation on the target during training and then applying the inverse transformation during prediction

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.compose import TransformedTargetRegressor
from numpy import nan as NA


# Feature engineering - Construct total compensation
df['Yearly brutto salary (without bonus and stocks) in EUR'] = pd.to_numeric(df['Yearly brutto salary (without bonus and stocks) in EUR'], errors='coerce')
df['Yearly bonus + stocks in EUR'] = pd.to_numeric(df['Yearly bonus + stocks in EUR'], errors='coerce')

# Drop rows with missing values in the columns used for calculation
df = df.dropna(subset=['Yearly brutto salary (without bonus and stocks) in EUR', 'Yearly bonus + stocks in EUR'])

# Feature engineering - Construct total compensation
df['Total Compensation'] = df['Yearly brutto salary (without bonus and stocks) in EUR'] + df['Yearly bonus + stocks in EUR']

# Split data into features and target variable
X = df.drop(['Total Compensation', 'Yearly brutto salary (without bonus and stocks) in EUR', 'Yearly bonus + stocks in EUR'], axis=1)
y = df['Total Compensation']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function for data preprocessing
def preprocess_data(df):
    # Handling missing values
    df['Yearly brutto salary (without bonus and stocks) in EUR'].fillna(df['Yearly brutto salary (without bonus and stocks) in EUR'].median(), inplace=True)
    df['Yearly bonus + stocks in EUR'].fillna(0, inplace=True)

    # Feature engineering if needed
    # ...

    # Categorical variable handling
    # You may need to customize this based on your dataset
    categorical_cols = [col for col in df.columns if df[col].dtype == 'object']
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
        ],
        remainder='passthrough'
    )

    # Define the model
    model = ExtraTreesRegressor()

    # TransformedTargetRegressor
    transformer = TransformedTargetRegressor(regressor=model, func='log1p', inverse_func='expm1')

    # Create pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', transformer)
    ])

    return pipeline

# Function for model evaluation
def evaluate_model(model, X_train, X_test, y_train, y_test):
    # Training the model
    model.fit(X_train, y_train)

    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Inverse transform for evaluation metrics
    y_train_pred = model.named_steps['model'].inverse_transform(y_train_pred.reshape(-1, 1)).flatten()
    y_test_pred = model.named_steps['model'].inverse_transform(y_test_pred.reshape(-1, 1)).flatten()

    # Evaluate using R2 and RMSE
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)

    rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)
    rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)

    return r2_train, r2_test, rmse_train, rmse_test

# Function for feature selection
def feature_selection(data, model):
    # Implement your feature selection technique here
    # Example: Remove features with low feature importance
    feature_importance = model.named_steps['model'].regressor_.feature_importances_
    selected_features = data.columns[feature_importance > threshold]
    return selected_features

# Function for hyperparameter tuning
def hyperparameter_tuning(model, param_grid, X_train, y_train):
    # Implement your hyperparameter tuning here
    grid_search = GridSearchCV(model, param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    return best_params

# Preprocess data and train the model
pipeline = preprocess_data(X_train)
r2_train, r2_test, rmse_train, rmse_test = evaluate_model(pipeline, X_train, X_test, y_train, y_test)

# Print results
print("### Model Evaluation ###")
print(f"R2 Score (Train): {r2_train}")
print(f"R2 Score (Test): {r2_test}")
print(f"RMSE (Train): {rmse_train}")
print(f"RMSE (Test): {rmse_test}")

# Feature selection
selected_features = feature_selection(X_train, pipeline)
print("\n### Feature Selection ###")
print(f"Selected Features: {selected_features}")

# Hyperparameter tuning
param_grid = {'model__regressor__n_estimators': [50, 100, 200], 'model__regressor__max_depth': [None, 10, 20]}
best_params = hyperparameter_tuning(pipeline, param_grid, X_train, y_train)
print("\n### Hyperparameter Tuning ###")
print(f"Best Hyperparameters: {best_params}")
